importing packages

In [1]:
import numpy as np
import string 
import scipy.io
import os
import re

import nilearn.signal

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Reading brain Data:

In [2]:
subject_id = "1"
datadir = "../data/"
subject_file = scipy.io.loadmat(datadir+'subject_'+subject_id+'.mat') 

sub_id_index = 0
#nTRs
number_of_scans_index = 1
#nvoxels
number_of_voxels = 2
x_dim_index = 3
y_dim_index = 4
z_dim_index = 5
#colToCoord
colToCoord_index = 6
#coordToCol
coordToCol_index = 7
ROInumToName_index = 8
ROInumsToName_3d_index = 9
ROINames_index = 10
voxel_size_index = 11
#the map to the MNI space
matrix_index = 12

activations = subject_file['data']
subject_id = subject_file['meta'][0][0][sub_id_index][0][0]
number_of_scans = subject_file['meta'][0][0][number_of_scans_index][0][0]
number_of_voxels = subject_file['meta'][0][0][number_of_voxels][0][0]
x_dim = subject_file['meta'][0][0][x_dim_index][0][0]
y_dim = subject_file['meta'][0][0][y_dim_index][0][0]
z_dim = subject_file['meta'][0][0][z_dim_index][0][0]
coords = subject_file['meta'][0][0][colToCoord_index]
named_areas = subject_file['meta'][0][0][ROInumToName_index]
column_map = subject_file['meta'][0][0][coordToCol_index].shape
area_names_list = subject_file['meta'][0][0][ROINames_index][0]
voxel_size = subject_file['meta'][0][0][voxel_size_index]
nmi_matrix = subject_file['meta'][0][0][matrix_index]

Reading Story Feature:

In [3]:
index = 0
word_index = 0
time_index = 1
subject_file['words'][0][index][word_index][0][0][0]
story_features = scipy.io.loadmat('../data/story_features.mat') 
part_of_speaches_feature_id = 8
speach_feature_id = 1
part_of_speaches = story_features['features'][0][part_of_speaches_feature_id][1][0]
part_of_speaches_features = story_features['features'][0][part_of_speaches_feature_id][2]
speaches = story_features['features'][0][speach_feature_id][1][0]
speach_features = story_features['features'][0][speach_feature_id][2]

In [4]:
print("How many words are there in the story? aligned with the brain data?")
subject_file['words'].shape

How many words are there in the story? aligned with the brain data?


(1, 5176)

In [5]:
actual_words = []
word_times = []
word_pos = []
speach_states = []
for i in np.arange(subject_file['words'].shape[1]):
    actual_words.append(subject_file['words'][0][i][word_index][0][0][0])
    word_times.append(subject_file['words'][0][i][time_index][0][0])
    word_pos.append(part_of_speaches_features[i])
    speach_states.append(speach_features[i])

Group time steps into blocks according to the fMRI experiment:

In [6]:
blocks = subject_file['time'][:,1]
scan_times = subject_file['time'][:,0]

block_ends = {}
block_ends_indexes = {}


for block_id in np.arange(1,5):
    block_ends_indexes[block_id] = np.where(scan_times == np.max(scan_times[np.where(blocks == block_id )]))[0]
    block_ends[block_id] = scan_times[block_ends_indexes[block_id]]+2


print(block_ends)
print(block_ends_indexes)

{1: array([680], dtype=uint16), 2: array([1384], dtype=uint16), 3: array([1942], dtype=uint16), 4: array([2702], dtype=uint16)}
{1: array([339]), 2: array([691]), 3: array([970]), 4: array([1350])}


group words and story features into blocks:

In [7]:
block_texts = {1:[],2:[],3:[],4:[]}
block_pos = {1:[],2:[],3:[],4:[]}
block_speach_state = {1:[],2:[],3:[],4:[]}
block_steps = {1:[],2:[],3:[],4:[]}
block_id = 1
for index in np.arange(len(actual_words)):
    if word_times[index] > block_ends[block_id]:
        block_id += 1
    block_texts[block_id].append(str(actual_words[index].encode("ascii",'ignore').decode()))
    block_pos[block_id].append(word_pos[index])
    block_speach_state[block_id].append(speach_states[index])
    block_steps[block_id].append(index)

In [8]:
len(list(zip(block_steps[1],block_texts[1])))

1303

Prepare input stimuli for the computational model (character input language model):

In [9]:
block_sentences = {1:[],2:[],3:[],4:[]}
translated_block_texts = {1:[],2:[],3:[],4:[]}
translated_block_pos = {1:[],2:[],3:[],4:[]}
translated_block_texts_steps = {1:[],2:[],3:[],4:[]}
translator = str.maketrans('', '', string.punctuation)


for block_id in np.arange(1,5):
    new_sentence = []
    for word,step,pos in zip(block_texts[block_id],block_steps[block_id],block_pos[block_id]):
        splitted_word = re.split("([A-Za-z\']+)", word.strip())
        tranlated_word = word.translate(translator)
        print("step:",step, "splited word:",splitted_word)
        last_w = ""
        for w in splitted_word:
            if len(w) > 0:
                last_w = w
                new_sentence.append(w)
                translated_block_texts[block_id].append(w)
                translated_block_texts_steps[block_id].append(step)
                translated_block_pos[block_id].append(pos)
        if last_w in ".?!":
            block_sentences[block_id].append(new_sentence)
            new_sentence = []
        
        """
        if len(word) == 1:
            tranlated_word = word
        if len(tranlated_word.strip()) > 0:
            translated_block_texts[block_id].append(tranlated_word)
            translated_block_texts_steps[block_id].append(step)
            translated_block_pos[block_id].append(pos)
            
        if len(tranlated_word.strip()) > 0:
            new_sentence.append(tranlated_word)
        if word.endswith("."):
            new_sentence.append(".")
            translated_block_texts[block_id].append(".")
            translated_block_texts_steps[block_id].append(step)
            translated_block_pos[block_id].append(np.zeros_like(pos))

            block_sentences[block_id].append(new_sentence)
            new_sentence = []
        """
    if len(new_sentence ) > 0:
            block_sentences[block_id].append(new_sentence)

    
    

step: 0 splited word: ['', 'Harry', '']
step: 1 splited word: ['', 'had', '']
step: 2 splited word: ['', 'never', '']
step: 3 splited word: ['', 'believed', '']
step: 4 splited word: ['', 'he', '']
step: 5 splited word: ['', 'would', '']
step: 6 splited word: ['', 'meet', '']
step: 7 splited word: ['', 'a', '']
step: 8 splited word: ['', 'boy', '']
step: 9 splited word: ['', 'he', '']
step: 10 splited word: ['', 'hated', '']
step: 11 splited word: ['', 'more', '']
step: 12 splited word: ['', 'than', '']
step: 13 splited word: ['', 'Dudley', ',']
step: 14 splited word: ['', 'but', '']
step: 15 splited word: ['', 'that', '']
step: 16 splited word: ['', 'was', '']
step: 17 splited word: ['', 'before', '']
step: 18 splited word: ['', 'he', '']
step: 19 splited word: ['', 'met', '']
step: 20 splited word: ['', 'Draco', '']
step: 21 splited word: ['', 'Malfoy', '.']
step: 22 splited word: ['', 'Still', ',']
step: 23 splited word: ['', 'first', '-', 'year', '']
step: 24 splited word: ['', 'Gr

step: 482 splited word: ['', 'grandmother', '.']
step: 483 splited word: ['', 'He', '']
step: 484 splited word: ['', 'opened', '']
step: 485 splited word: ['', 'it', '']
step: 486 splited word: ['', 'excitedly', '']
step: 487 splited word: ['', 'and', '']
step: 488 splited word: ['', 'showed', '']
step: 489 splited word: ['', 'them', '']
step: 490 splited word: ['', 'a', '']
step: 491 splited word: ['', 'glass', '']
step: 492 splited word: ['', 'ball', '']
step: 493 splited word: ['', 'the', '']
step: 494 splited word: ['', 'size', '']
step: 495 splited word: ['', 'of', '']
step: 496 splited word: ['', 'a', '']
step: 497 splited word: ['', 'large', '']
step: 498 splited word: ['', 'marble', ',']
step: 499 splited word: ['', 'which', '']
step: 500 splited word: ['', 'seemed', '']
step: 501 splited word: ['', 'to', '']
step: 502 splited word: ['', 'be', '']
step: 503 splited word: ['', 'full', '']
step: 504 splited word: ['', 'of', '']
step: 505 splited word: ['', 'white', '']
step: 506 

step: 857 splited word: ['', 'jumped', '']
step: 858 splited word: ['', 'into', '']
step: 859 splited word: ['', 'his', '']
step: 860 splited word: ['', 'hand', '']
step: 861 splited word: ['', 'at', '']
step: 862 splited word: ['', 'once', ',']
step: 863 splited word: ['', 'but', '']
step: 864 splited word: ['', 'it', '']
step: 865 splited word: ['', 'was', '']
step: 866 splited word: ['', 'one', '']
step: 867 splited word: ['', 'of', '']
step: 868 splited word: ['', 'the', '']
step: 869 splited word: ['', 'few', '']
step: 870 splited word: ['', 'that', '']
step: 871 splited word: ['', 'did', '.']
step: 872 splited word: ['', 'Hermione', '']
step: 873 splited word: ['', "Granger's", '']
step: 874 splited word: ['', 'had', '']
step: 875 splited word: ['', 'simply', '']
step: 876 splited word: ['', 'rolled', '']
step: 877 splited word: ['', 'over', '']
step: 878 splited word: ['', 'on', '']
step: 879 splited word: ['', 'the', '']
step: 880 splited word: ['', 'ground', ',']
step: 881 spl

step: 1194 splited word: ['+']
step: 1195 splited word: ['', 'Neville', ',']
step: 1196 splited word: ['', 'his', '']
step: 1197 splited word: ['', 'face', '']
step: 1198 splited word: ['', 'tear', '-', 'streaked', ',']
step: 1199 splited word: ['', 'clutching', '']
step: 1200 splited word: ['', 'his', '']
step: 1201 splited word: ['', 'wrist', ',']
step: 1202 splited word: ['', 'hobbled', '']
step: 1203 splited word: ['', 'off', '']
step: 1204 splited word: ['', 'with', '']
step: 1205 splited word: ['', 'Madam', '']
step: 1206 splited word: ['', 'Hooch', ',']
step: 1207 splited word: ['', 'who', '']
step: 1208 splited word: ['', 'had', '']
step: 1209 splited word: ['', 'her', '']
step: 1210 splited word: ['', 'arm', '']
step: 1211 splited word: ['', 'around', '']
step: 1212 splited word: ['', 'him', '.']
step: 1213 splited word: ['+']
step: 1214 splited word: ['', 'No', '']
step: 1215 splited word: ['', 'sooner', '']
step: 1216 splited word: ['', 'were', '']
step: 1217 splited word: [

step: 1607 splited word: ['', 'it', '']
step: 1608 splited word: ['', 'if', '']
step: 1609 splited word: ['', 'you', '']
step: 1610 splited word: ['', 'can', ',']
step: 1611 splited word: ['', 'then', '!"']
step: 1612 splited word: ['', 'he', '']
step: 1613 splited word: ['', 'shouted', ',']
step: 1614 splited word: ['', 'and', '']
step: 1615 splited word: ['', 'he', '']
step: 1616 splited word: ['', 'threw', '']
step: 1617 splited word: ['', 'the', '']
step: 1618 splited word: ['', 'glass', '']
step: 1619 splited word: ['', 'ball', '']
step: 1620 splited word: ['', 'high', '']
step: 1621 splited word: ['', 'into', '']
step: 1622 splited word: ['', 'the', '']
step: 1623 splited word: ['', 'air', '']
step: 1624 splited word: ['', 'and', '']
step: 1625 splited word: ['', 'streaked', '']
step: 1626 splited word: ['', 'back', '']
step: 1627 splited word: ['', 'toward', '']
step: 1628 splited word: ['', 'the', '']
step: 1629 splited word: ['', 'ground', '.']
step: 1630 splited word: ['+']
s

...

step: 3481 splited word: ['', 'even', '']
step: 3482 splited word: ['', 'reached', '']
step: 3483 splited word: ['', 'the', '']
step: 3484 splited word: ['', 'end', '']
step: 3485 splited word: ['', 'of', '']
step: 3486 splited word: ['', 'the', '']
step: 3487 splited word: ['', 'corridor', '']
step: 3488 splited word: ['', 'when', '']
step: 3489 splited word: ['', 'Hermione', '']
step: 3490 splited word: ['', 'caught', '']
step: 3491 splited word: ['', 'up', '']
step: 3492 splited word: ['', 'with', '']
step: 3493 splited word: ['', 'them', '.']
step: 3494 splited word: ['+']
step: 3495 splited word: ['"', "I'm", '']
step: 3496 splited word: ['', 'coming', '']
step: 3497 splited word: ['', 'with', '']
step: 3498 splited word: ['', 'you', ',"']
step: 3499 splited word: ['', 'she', '']
step: 3500 splited word: ['', 'said', '.']
step: 3501 splited word: ['"', 'You', '']
step: 3502 splited word: ['', 'are', '']
step: 3503 splited word: ['@', 'not', '."']
step: 3504 splited word: ['+']
ste

step: 3981 splited word: ['', 'round', '']
step: 3982 splited word: ['', 'the', '']
step: 3983 splited word: ['', 'corner', '']
step: 3984 splited word: ['', 'when', '']
step: 3985 splited word: ['', 'they', '']
step: 3986 splited word: ['', 'heard', '']
step: 3987 splited word: ['', 'Filch', '']
step: 3988 splited word: ['', 'enter', '']
step: 3989 splited word: ['', 'the', '']
step: 3990 splited word: ['', 'trophy', '']
step: 3991 splited word: ['', 'room', '.']
step: 3992 splited word: ['+']
step: 3993 splited word: ['"', "They're", '']
step: 3994 splited word: ['', 'in', '']
step: 3995 splited word: ['', 'here', '']
step: 3996 splited word: ['', 'somewhere', ',"']
step: 3997 splited word: ['', 'they', '']
step: 3998 splited word: ['', 'heard', '']
step: 3999 splited word: ['', 'him', '']
step: 4000 splited word: ['', 'mutter', ',']
step: 4001 splited word: ['"', 'probably', '']
step: 4002 splited word: ['', 'hiding', '."']
step: 4003 splited word: ['+']
step: 4004 splited word: ['"

step: 4356 splited word: ['', 'us', '']
step: 4357 splited word: ['', 'away', ',']
step: 4358 splited word: ['', 'Peeves', ',']
step: 4359 splited word: ['', 'please', '."']
step: 4360 splited word: ['+']
step: 4361 splited word: ['"', 'Should', '']
step: 4362 splited word: ['', 'tell', '']
step: 4363 splited word: ['', 'Filch', ',']
step: 4364 splited word: ['', 'I', '']
step: 4365 splited word: ['', 'should', ',"']
step: 4366 splited word: ['', 'said', '']
step: 4367 splited word: ['', 'Peeves', '']
step: 4368 splited word: ['', 'in', '']
step: 4369 splited word: ['', 'a', '']
step: 4370 splited word: ['', 'saintly', '']
step: 4371 splited word: ['', 'voice', ',']
step: 4372 splited word: ['', 'but', '']
step: 4373 splited word: ['', 'his', '']
step: 4374 splited word: ['', 'eyes', '']
step: 4375 splited word: ['', 'glittered', '']
step: 4376 splited word: ['', 'wickedly', '.']
step: 4377 splited word: ['"', "It's", '']
step: 4378 splited word: ['', 'for', '']
step: 4379 splited word

step: 4731 splited word: ['', 'twitching', '']
step: 4732 splited word: ['', 'and', '']
step: 4733 splited word: ['', 'quivering', '']
step: 4734 splited word: ['', 'in', '']
step: 4735 splited word: ['', 'their', '']
step: 4736 splited word: ['', 'direction', ';']
step: 4737 splited word: ['', 'three', '']
step: 4738 splited word: ['', 'drooling', '']
step: 4739 splited word: ['', 'mouths', ',']
step: 4740 splited word: ['', 'saliva', '']
step: 4741 splited word: ['', 'hanging', '']
step: 4742 splited word: ['', 'in', '']
step: 4743 splited word: ['', 'slippery', '']
step: 4744 splited word: ['', 'ropes', '']
step: 4745 splited word: ['', 'from', '']
step: 4746 splited word: ['', 'yellowish', '']
step: 4747 splited word: ['', 'fangs', '.']
step: 4748 splited word: ['+']
step: 4749 splited word: ['', 'It', '']
step: 4750 splited word: ['', 'was', '']
step: 4751 splited word: ['', 'standing', '']
step: 4752 splited word: ['', 'quite', '']
step: 4753 splited word: ['', 'still', ',']
step

step: 5106 splited word: ['', 'you', '?"']
step: 5107 splited word: ['+']
step: 5108 splited word: ['', 'But', '']
step: 5109 splited word: ['', 'Hermione', '']
step: 5110 splited word: ['', 'had', '']
step: 5111 splited word: ['', 'given', '']
step: 5112 splited word: ['', 'Harry', '']
step: 5113 splited word: ['', 'something', '']
step: 5114 splited word: ['', 'else', '']
step: 5115 splited word: ['', 'to', '']
step: 5116 splited word: ['', 'think', '']
step: 5117 splited word: ['', 'about', '']
step: 5118 splited word: ['', 'as', '']
step: 5119 splited word: ['', 'he', '']
step: 5120 splited word: ['', 'climbed', '']
step: 5121 splited word: ['', 'back', '']
step: 5122 splited word: ['', 'into', '']
step: 5123 splited word: ['', 'bed', '.']
step: 5124 splited word: ['', 'The', '']
step: 5125 splited word: ['', 'dog', '']
step: 5126 splited word: ['', 'was', '']
step: 5127 splited word: ['', 'guarding', '']
step: 5128 splited word: ['', 'something', '.']
step: 5129 splited word: ['.'

In [10]:
block_sentences[1] 

[['Harry',
  'had',
  'never',
  'believed',
  'he',
  'would',
  'meet',
  'a',
  'boy',
  'he',
  'hated',
  'more',
  'than',
  'Dudley',
  ',',
  'but',
  'that',
  'was',
  'before',
  'he',
  'met',
  'Draco',
  'Malfoy',
  '.'],
 ['Still',
  ',',
  'first',
  '-',
  'year',
  'Gryffindors',
  'only',
  'had',
  'Potions',
  'with',
  'the',
  'Slytherins',
  ',',
  'so',
  'they',
  "didn't",
  'have',
  'to',
  'put',
  'up',
  'with',
  'Malfoy',
  'much',
  '.'],
 ['Or',
  'at',
  'least',
  ',',
  'they',
  "didn't",
  'until',
  'they',
  'spotted',
  'a',
  'notice',
  'pinned',
  'up',
  'in',
  'the',
  'Gryffindor',
  'common',
  'room',
  'that',
  'made',
  'them',
  'all',
  'groan',
  '.'],
 ['Flying',
  'lessons',
  'would',
  'be',
  'starting',
  'on',
  'Thursday',
  '--',
  'and',
  'Gryffindor',
  'and',
  'Slytherin',
  'would',
  'be',
  'learning',
  'together',
  '.'],
 ['+', '"', 'Typical', ',"', 'said', 'Harry', 'darkly', '.'],
 ['"', 'Just', 'what', 'I'

In [11]:
block_text = {}
for block_id in np.arange(1,5):
    block_text[block_id] = ' '.join(translated_block_texts[block_id])
    
print(block_text)

{1: 'Harry had never believed he would meet a boy he hated more than Dudley , but that was before he met Draco Malfoy . Still , first - year Gryffindors only had Potions with the Slytherins , so they didn\'t have to put up with Malfoy much . Or at least , they didn\'t until they spotted a notice pinned up in the Gryffindor common room that made them all groan . Flying lessons would be starting on Thursday -- and Gryffindor and Slytherin would be learning together . + " Typical ," said Harry darkly . " Just what I always wanted . To make a fool of myself on a broomstick in front of Malfoy ." + He had been looking forward to learning to fly more than anything else . " You don\'t know that you\'ll make a fool of yourself ," said Ron reasonably . " Anyway , I know Malfoy\'s always going on about how good he is at Quidditch , but I bet that\'s all talk ." + Malfoy certainly did talk about flying a lot . He complained loudly about first years never getting on the House Quidditch teams and to

In [12]:
processed_data_dir = "../processed_data/subject_"+str(subject_id)
try:
    os.stat(processed_data_dir)
except:
    os.makedirs(processed_data_dir) 
for block_id in np.arange(1,5):
    
    np.save(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_sentences",block_sentences[block_id])
    np.save(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_text",translated_block_texts[block_id])
    np.save(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_pos",translated_block_pos[block_id])

    with open(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_rawtext.txt","w") as fout:
        fout.write(block_text[block_id])

In [13]:
print(np.asarray(translated_block_texts[1])[np.where(np.asarray(translated_block_texts_steps[1]) == 117)])
print(np.asarray(translated_block_texts_steps[1])[np.where(np.asarray(translated_block_texts_steps[1]) == 117)])

["don't"]
[117]


In [14]:
translated_block_texts_steps[1]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 21,
 22,
 22,
 23,
 23,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 40,
 41,
 42,
 43,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 78,
 79,
 80,
 80,
 80,
 81,
 82,
 83,
 83,
 84,
 84,
 85,
 86,
 87,
 88,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 115,
 116,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 125,
 126,
 127,
 128,
 128,
 129,
 129,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 157,
 158,
 159,
 160,
 161,
 16

In [15]:
class Scan(object):
    def __init__(self,activations,timestamp, step,prev_words=None,next_words=None,all_words=None,all_pos=None,all_speak_features=None,current_translated_words=None):
        self.activations = activations
        self.timestamp = timestamp
        self.prev_words = prev_words
        self.next_words = next_words
        self.step = step
        self.all_words = all_words
        self.current_translated_words = current_translated_words
        self.all_pos = all_pos
        self.all_speak_features = all_speak_features
        self.brain3d = None
        


In [16]:
vocab = []
translated_words = []
with open(processed_data_dir+"/subject_"+subject_id+"_harry_potter_vocab.txt","w") as fout:
    for word in actual_words:
        w = word.encode("ascii",'ignore').decode()
        if len(w) > 1:
            w = w #.translate(translator)
        if len(w) > 0:
            fout.write(w+'\n')
            print(w)
            vocab.append(w)

        translated_words.append(w)
        

Harry
had
never
believed
he
would
meet
a
boy
he
hated
more
than
Dudley,
but
that
was
before
he
met
Draco
Malfoy.
Still,
first-year
Gryffindors
only
had
Potions
with
the
Slytherins,
so
they
didn't
have
to
put
up
with
Malfoy
much.
Or
at
least,
they
didn't
until
they
spotted
a
notice
pinned
up
in
the
Gryffindor
common
room
that
made
them
all
groan.
Flying
lessons
would
be
starting
on
Thursday
--
and
Gryffindor
and
Slytherin
would
be
learning
together.
+
"Typical,"
said
Harry
darkly.
"Just
what
I
always
wanted.
To
make
a
fool
of
myself
on
a
broomstick
in
front
of
Malfoy."
+
He
had
been
looking
forward
to
learning
to
fly
more
than
anything
else.
"You
don't
know
that
you'll
make
a
fool
of
yourself,"
said
Ron
reasonably.
"Anyway,
I
know
Malfoy's
always
going
on
about
how
good
he
is
at
Quidditch,
but
I
bet
that's
all
talk."
+
Malfoy
certainly
did
talk
about
flying
a
lot.
He
complained
loudly
about
first
years
never
getting
on
the
House
Quidditch
teams
and
told
long,
boastful
stories
that
alway

--
wind
whistled
in
his
ears,
mingled
with
the
screams
of
people
watching
--
he
stretched
out
his
hand
--
a
foot
from
the
ground
he
caught
it,
just
in
time
to
pull
his
broom
straight,
and
he
toppled
gently
onto
the
grass
with
the
Remembrall
clutched
safely
in
his
fist.
+
"HARRY
POTTER!"
+
His
heart
sank
faster
than
he'd
just
dived.
Professor
McGonagall
was
running
toward
them.
He
got
to
his
feet,
trembling.
+
@"Never
--
in
all
my
time
at
Hogwarts
"
Professor
McGonagall
was
almost
speechless
with
shock,
and
her
glasses
flashed
furiously,
"
how
@dare
you
--
might
have
broken
your
neck
"
+
"It
wasn't
his
fault,
Professor
"
+
"Be
quiet,
Miss
Patil
"
+
"But
Malfoy
"
"That's
enough,
Mr.
Weasley.
Potter,
follow
me,
now."
+
Harry
caught
sight
of
Malfoy,
Crabbe,
and
Goyle's
triumphant
faces
as
he
left,
walking
numbly
in
Professor
McGonagall's
wake
as
she
strode
toward
the
castle.
He
was
going
to
be
expelled,
he
just
knew
it.
He
wanted
to
say
something
to
defend
himself,
but
there
seemed
to
be
s

muttered
at
last,
"we'd
better
go."
+
They
pulled
on
their
bathrobes,
picked
up
their
wands,
and
crept
across
the
tower
room,
down
the
spiral
staircase,
and
into
the
Gryffindor
common
room.
A
few
embers
were
still
glowing
in
the
fireplace,
turning
all
the
armchairs
into
hunched
black
shadows.
They
had
almost
reached
the
portrait
hole
when
a
voice
spoke
from
the
chair
nearest
them,
"I
can't
believe
you're
going
to
do
this,
Harry."
+
A
lamp
flickered
on.
It
was
Hermione
Granger,
wearing
a
pink
bathrobe
and
a
frown.
+
@"You!"
said
Ron
furiously.
"Go
back
to
bed!"
+
"I
almost
told
your
brother,"
Hermione
snapped,
"Percy
--
he's
a
prefect,
he'd
put
a
stop
to
this."
+
Harry
couldn't
believe
anyone
could
be
so
interfering.
+
"Come
on,"
he
said
to
Ron.
He
pushed
open
the
portrait
of
the
Fat
Lady
and
climbed
through
the
hole.
+
Hermione
wasn't
going
to
give
up
that
easily.
She
followed
Ron
through
the
portrait
hole,
hissing
at
them
like
an
angry
goose.
"Don't
you
@care
about
Gryffindor,
do
you


In [17]:
len(word_pos)
len(translated_words)
processed_data_dir

'../processed_data/subject_1'

Detrending brain activations:

In [20]:
block_activations = {1:[],2:[],3:[],4:[]}
block_id = 1
for j in np.arange(0,len(activations)):
    if j > block_ends_indexes[block_id][0]:
        block_id +=1
    block_activations[block_id].append(activations[j])

print(len(block_activations[1]))
print(len(block_activations[2]))
print(len(block_activations[3]))
print(len(block_activations[4]))

clean_block_activations = {1:[],2:[],3:[],4:[]}
all_clean_activations = []
for block_id in block_activations.keys():
    print("block id:",block_id)

    clean_activations = nilearn.signal.clean(block_activations[block_id], sessions=None, 
                         detrend=True, standardize=True, 
                         confounds=None, low_pass=None, 
                         high_pass=0.005, t_r=2.0, ensure_finite=False)
    clean_block_activations[block_id] = clean_activations
    all_clean_activations.extend(clean_activations)

all_clean_activations = np.asarray(all_clean_activations)
print(len(all_clean_activations))

print("min:",np.min(all_clean_activations,axis=1))
print("max:",np.max(all_clean_activations,axis=1))

340
352
279
380
block id: 1
block id: 2
block id: 3
block id: 4
1351
min: [-0.6567336 -5.3272805 -5.1683035 ... -7.115965  -8.355176  -1.6093863]
max: [0.7477386 4.6919174 5.231791  ... 6.7115455 7.439363  1.5097184]


Saving Scan Objects:

In [21]:
block_scans = {1:[],2:[],3:[],4:[]}
block_id = 1

j = np.where(scan_times == word_times[0])[0]
for i in np.arange(0,len(actual_words)):
    if word_times[i] > scan_times[j]:
        j = np.where(scan_times == word_times[i])[0]
    print(block_id,word_times[i], scan_times[j][0])
    #print(word_times[i], scan_times[j])
    if word_times[i] == scan_times[j][0]:
        print(i,j)
        
        k = i
        current_step_words = []
        current_step_pos = []
        current_step_speach_states = []
        translated_current_words = []
        counter = 0
        while k < len(actual_words) and counter < 4 :
            current_step_words.append(actual_words[k])
            current_step_pos.append(word_pos[k])
            current_step_speach_states.append(speach_states[k])
            
            translated_text_index = np.where(np.asarray(translated_block_texts_steps[block_id]) == k)
            print(translated_text_index)
            translated_current_words.append(np.asarray(translated_block_texts[block_id])[translated_text_index])
            
            k += 1
            counter +=1
            
            
        k = i+4
        next_step_words = []
        next_step_pos = []
        next_step_speach_states = []
        counter = 0
        while k >= 0 and k < len(actual_words) and counter < 4 :
            next_step_words.append(actual_words[k])
            next_step_pos.append(word_pos[k])
            next_step_speach_states.append(speach_states[k])
            k += 1
            counter +=1
         
        k = i-4
        prev_step_words = []
        prev_step_pos = []
        prev_step_speach_states = []
        counter = 0
        while k >= 0 and k < len(actual_words) and counter < 4 :
            prev_step_words.append(actual_words[k])
            prev_step_pos.append(word_pos[k])
            prev_step_speach_states.append(speach_states[k])
            k += 1
            counter +=1
            
            
        print(current_step_words)
        print(translated_current_words)
        scan = Scan(all_clean_activations[j],scan_times[j][0],i,
                        prev_step_words,
                        next_step_words,
                        current_step_words,
                        current_step_pos,
                        current_step_speach_states,
                        translated_current_words) 
                   
        
        # print(i,j,block_ends_indexes[block_id][0])
        if j > block_ends_indexes[block_id][0]:
            block_id +=1
        j += 1
        block_scans[block_id].append(scan)
        


1 20 20
0 [10]
(array([0]),)
(array([1]),)
(array([2]),)
(array([3]),)
['Harry', 'had', 'never', 'believed']
[array(['Harry'], dtype='<U13'), array(['had'], dtype='<U13'), array(['never'], dtype='<U13'), array(['believed'], dtype='<U13')]
1 20.5 22
1 21 22
1 21.5 22
1 22 22
4 [11]
(array([4]),)
(array([5]),)
(array([6]),)
(array([7]),)
['he', 'would', 'meet', 'a']
[array(['he'], dtype='<U13'), array(['would'], dtype='<U13'), array(['meet'], dtype='<U13'), array(['a'], dtype='<U13')]
1 22.5 24
1 23 24
1 23.5 24
1 24 24
8 [12]
(array([8]),)
(array([9]),)
(array([10]),)
(array([11]),)
['boy', 'he', 'hated', 'more']
[array(['boy'], dtype='<U13'), array(['he'], dtype='<U13'), array(['hated'], dtype='<U13'), array(['more'], dtype='<U13')]
1 24.5 26
1 25 26
1 25.5 26
1 26 26
12 [13]
(array([12]),)
(array([13, 14]),)
(array([15]),)
(array([16]),)
['than', 'Dudley,', 'but', 'that']
[array(['than'], dtype='<U13'), array(['Dudley', ','], dtype='<U13'), array(['but'], dtype='<U13'), array(['that']

1 114 114
188 [57]
(array([215]),)
(array([216]),)
(array([217]),)
(array([218]),)
['He', "wasn't", 'the', 'only']
[array(['He'], dtype='<U13'), array(["wasn't"], dtype='<U13'), array(['the'], dtype='<U13'), array(['only'], dtype='<U13')]
1 114.5 116
1 115 116
1 115.5 116
1 116 116
192 [58]
(array([219, 220]),)
(array([221, 222]),)
(array([223]),)
(array([224]),)
['one,', 'though:', 'the', 'way']
[array(['one', ','], dtype='<U13'), array(['though', ':'], dtype='<U13'), array(['the'], dtype='<U13'), array(['way'], dtype='<U13')]
1 116.5 118
1 117 118
1 117.5 118
1 118 118
196 [59]
(array([225]),)
(array([226]),)
(array([227]),)
(array([228, 229]),)
['Seamus', 'Finnigan', 'told', 'it,']
[array(['Seamus'], dtype='<U13'), array(['Finnigan'], dtype='<U13'), array(['told'], dtype='<U13'), array(['it', ','], dtype='<U13')]
1 118.5 120
1 119 120
1 119.5 120
1 120 120
200 [60]
(array([230]),)
(array([231]),)
(array([232]),)
(array([233]),)
["he'd", 'spent', 'most', 'of']
[array(["he'd"], dtype=

(array([431]),)
(array([432]),)
(array([433]),)
['of', 'a', 'library', 'book']
[array(['of'], dtype='<U13'), array(['a'], dtype='<U13'), array(['library'], dtype='<U13'), array(['book'], dtype='<U13')]
1 212.5 214
1 213 214
1 213.5 214
1 214 214
388 [107]
(array([434]),)
(array([435, 436]),)
(array([437, 438]),)
(array([439, 440]),)
['called', '@Quidditch', '@Through', '@the']
[array(['called'], dtype='<U13'), array(['@', 'Quidditch'], dtype='<U13'), array(['@', 'Through'], dtype='<U13'), array(['@', 'the'], dtype='<U13')]
1 214.5 216
1 215 216
1 215.5 216
1 216 216
392 [108]
(array([441, 442, 443]),)
(array([444]),)
(array([445]),)
(array([446]),)
['@Ages.', 'Neville', 'was', 'hanging']
[array(['@', 'Ages', '.'], dtype='<U13'), array(['Neville'], dtype='<U13'), array(['was'], dtype='<U13'), array(['hanging'], dtype='<U13')]
1 216.5 218
1 217 218
1 217.5 218
1 218 218
396 [109]
(array([447]),)
(array([448]),)
(array([449]),)
(array([450]),)
['on', 'to', 'her', 'every']
[array(['on'], d

(array([612]),)
(array([613]),)
(array([614, 615]),)
(array([616]),)
['His', 'face', 'fell,', 'because']
[array(['His'], dtype='<U13'), array(['face'], dtype='<U13'), array(['fell', ','], dtype='<U13'), array(['because'], dtype='<U13')]
1 292.5 294
1 293 294
1 293.5 294
1 294 294
548 [147]
(array([617]),)
(array([618]),)
(array([619]),)
(array([620]),)
['the', 'Remembrall', 'had', 'suddenly']
[array(['the'], dtype='<U13'), array(['Remembrall'], dtype='<U13'), array(['had'], dtype='<U13'), array(['suddenly'], dtype='<U13')]
1 294.5 296
1 295 296
1 295.5 296
1 296 296
552 [148]
(array([621]),)
(array([622, 623]),)
(array([624]),)
(array([625]),)
['glowed', 'scarlet,', '"…', "you've"]
[array(['glowed'], dtype='<U13'), array(['scarlet', ','], dtype='<U13'), array(['"'], dtype='<U13'), array(["you've"], dtype='<U13')]
1 296.5 298
1 297 298
1 297.5 298
1 298 298
556 [149]
(array([626]),)
(array([627]),)
(array([628]),)
(array([629]),)
['forgotten', 'something', '…"', '+']
[array(['forgotten'

['front', 'steps', 'onto', 'the']
[array(['front'], dtype='<U13'), array(['steps'], dtype='<U13'), array(['onto'], dtype='<U13'), array(['the'], dtype='<U13')]
1 356.5 358
1 357 358
1 357.5 358
1 358 358
676 [179]
(array([770]),)
(array([771]),)
(array([772]),)
(array([773]),)
['grounds', 'for', 'their', 'first']
[array(['grounds'], dtype='<U13'), array(['for'], dtype='<U13'), array(['their'], dtype='<U13'), array(['first'], dtype='<U13')]
1 358.5 360
1 359 360
1 359.5 360
1 360 360
680 [180]
(array([774]),)
(array([775, 776]),)
(array([777]),)
(array([778]),)
['flying', 'lesson.', 'It', 'was']
[array(['flying'], dtype='<U13'), array(['lesson', '.'], dtype='<U13'), array(['It'], dtype='<U13'), array(['was'], dtype='<U13')]
1 360.5 362
1 361 362
1 361.5 362
1 362 362
684 [181]
(array([779]),)
(array([780, 781]),)
(array([782]),)
(array([783, 784]),)
['a', 'clear,', 'breezy', 'day,']
[array(['a'], dtype='<U13'), array(['clear', ','], dtype='<U13'), array(['breezy'], dtype='<U13'), array(

...

(array([891]),)
(array([892]),)
(array([893]),)
(array([894, 895]),)
['This', 'is', 'the', 'end!"']
[array(['This'], dtype='<U11'), array(['is'], dtype='<U11'), array(['the'], dtype='<U11'), array(['end', '!"'], dtype='<U11')]
4 2336.5 2338
4 2337 2338
4 2337.5 2338
4 2338 2338
4465 [1169]
(array([896]),)
(array([897]),)
(array([898]),)
(array([899]),)
['+', 'They', 'could', 'hear']
[array(['+'], dtype='<U11'), array(['They'], dtype='<U11'), array(['could'], dtype='<U11'), array(['hear'], dtype='<U11')]
4 2338.5 2340
4 2339 2340
4 2339.5 2340
4 2340 2340
4469 [1170]
(array([900, 901]),)
(array([902]),)
(array([903]),)
(array([904]),)
['footsteps,', 'Filch', 'running', 'as']
[array(['footsteps', ','], dtype='<U11'), array(['Filch'], dtype='<U11'), array(['running'], dtype='<U11'), array(['as'], dtype='<U11')]
4 2340.5 2342
4 2341 2342
4 2341.5 2342
4 2342 2342
4473 [1171]
(array([905]),)
(array([906]),)
(array([907]),)
(array([908]),)
['fast', 'as', 'he', 'could']
[array(['fast'], dtype

4 2410.5 2412
4 2411 2412
4 2411.5 2412
4 2412 2412
4613 [1206]
(array([1095, 1096, 1097]),)
(array([1098, 1099]),)
(array([1100]),)
(array([1101]),)
['@off,', 'Neville!"', 'For', 'Neville']
[array(['@', 'off', ','], dtype='<U11'), array(['Neville', '!"'], dtype='<U11'), array(['For'], dtype='<U11'), array(['Neville'], dtype='<U11')]
4 2412.5 2414
4 2413 2414
4 2413.5 2414
4 2414 2414
4617 [1207]
(array([1102]),)
(array([1103]),)
(array([1104]),)
(array([1105]),)
['had', 'been', 'tugging', 'on']
[array(['had'], dtype='<U11'), array(['been'], dtype='<U11'), array(['tugging'], dtype='<U11'), array(['on'], dtype='<U11')]
4 2414.5 2416
4 2415 2416
4 2415.5 2416
4 2416 2416
4621 [1208]
(array([1106]),)
(array([1107]),)
(array([1108]),)
(array([1109]),)
['the', 'sleeve', 'of', "Harry's"]
[array(['the'], dtype='<U11'), array(['sleeve'], dtype='<U11'), array(['of'], dtype='<U11'), array(["Harry's"], dtype='<U11')]
4 2416.5 2418
4 2417 2418
4 2417.5 2418
4 2418 2418
4625 [1209]
(array([1110]),)

4 2496.5 2498
4 2497 2498
4 2497.5 2498
4 2498 2498
4785 [1249]
(array([1296]),)
(array([1297]),)
(array([1298, 1299]),)
(array([1300]),)
['getting', 'over', 'that,', 'there']
[array(['getting'], dtype='<U11'), array(['over'], dtype='<U11'), array(['that', ','], dtype='<U11'), array(['there'], dtype='<U11')]
4 2498.5 2500
4 2499 2500
4 2499.5 2500
4 2500 2500
4789 [1250]
(array([1301]),)
(array([1302]),)
(array([1303]),)
(array([1304]),)
['was', 'no', 'mistaking', 'what']
[array(['was'], dtype='<U11'), array(['no'], dtype='<U11'), array(['mistaking'], dtype='<U11'), array(['what'], dtype='<U11')]
4 2500.5 2502
4 2501 2502
4 2501.5 2502
4 2502 2502
4793 [1251]
(array([1305]),)
(array([1306]),)
(array([1307]),)
(array([1308, 1309]),)
['those', 'thunderous', 'growls', 'meant.']
[array(['those'], dtype='<U11'), array(['thunderous'], dtype='<U11'), array(['growls'], dtype='<U11'), array(['meant', '.'], dtype='<U11')]
4 2502.5 2504
4 2503 2504
4 2503.5 2504
4 2504 2504
4797 [1252]
(array([13

4 2591 2592
4 2591.5 2592
4 2592 2592
4973 [1296]
(array([1515, 1516]),)
(array([1517]),)
(array([1518]),)
(array([1519, 1520]),)
['school?"', 'said', 'Ron', 'finally.']
[array(['school', '?"'], dtype='<U11'), array(['said'], dtype='<U11'), array(['Ron'], dtype='<U11'), array(['finally', '.'], dtype='<U11')]
4 2592.5 2594
4 2593 2594
4 2593.5 2594
4 2594 2594
4977 [1297]
(array([1521, 1522]),)
(array([1523]),)
(array([1524]),)
(array([1525]),)
['"If', 'any', 'dog', 'needs']
[array(['"', 'If'], dtype='<U11'), array(['any'], dtype='<U11'), array(['dog'], dtype='<U11'), array(['needs'], dtype='<U11')]
4 2594.5 2596
4 2595 2596
4 2595.5 2596
4 2596 2596
4981 [1298]
(array([1526, 1527]),)
(array([1528]),)
(array([1529]),)
(array([1530, 1531]),)
['exercise,', 'that', 'one', 'does."']
[array(['exercise', ','], dtype='<U11'), array(['that'], dtype='<U11'), array(['one'], dtype='<U11'), array(['does', '."'], dtype='<U11')]
4 2596.5 2598
4 2597 2598
4 2597.5 2598
4 2598 2598
4985 [1299]
(array([

5161 [1343]
(array([1750]),)
(array([1751]),)
(array([1752]),)
(array([1753]),)
['found', 'out', 'where', 'the']
[array(['found'], dtype='<U11'), array(['out'], dtype='<U11'), array(['where'], dtype='<U11'), array(['the'], dtype='<U11')]
4 2686.5 2688
4 2687 2688
4 2687.5 2688
4 2688 2688
5165 [1344]
(array([1754]),)
(array([1755]),)
(array([1756]),)
(array([1757]),)
['grubby', 'little', 'package', 'from']
[array(['grubby'], dtype='<U11'), array(['little'], dtype='<U11'), array(['package'], dtype='<U11'), array(['from'], dtype='<U11')]
4 2688.5 2690
4 2689 2690
4 2689.5 2690
4 2690 2690
5169 [1345]
(array([1758]),)
(array([1759]),)
(array([1760]),)
(array([1761]),)
['vault', 'seven', 'hundred', 'and']
[array(['vault'], dtype='<U11'), array(['seven'], dtype='<U11'), array(['hundred'], dtype='<U11'), array(['and'], dtype='<U11')]
4 2690.5 2692
4 2691 2692
4 2691.5 2692
4 2692 2692
5173 [1346]
(array([1762]),)
(array([1763, 1764]),)
(array([1765]),)
['thirteen', 'was.', '+']
[array(['thir

In [22]:
block_scans[1][50].all_words

["he'd", 'spent', 'most', 'of']

In [23]:
translated_block_texts_steps[1]
print(len(block_text[4].split()))
print(len(translated_block_texts[4]))

for i in np.arange(len(block_text[1].split())):
    if block_text[1].split()[i] != translated_block_texts[1][i]:
        print(block_text[1].split()[i],translated_block_texts[1][i])


1766
1766


In [24]:
block_scans[4][0].all_words

['Ron', 'looked', 'at', 'his']

In [25]:
np.save(processed_data_dir+"/subject_"+subject_id+"_scan_objects",block_scans)

In [26]:
block_id = 1
print(translated_block_texts_steps[block_id])
np.where(np.asarray(translated_block_texts_steps[block_id]) == 1)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 13, 14, 15, 16, 17, 18, 19, 20, 21, 21, 22, 22, 23, 23, 23, 24, 25, 26, 27, 28, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 40, 41, 42, 43, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 78, 79, 80, 80, 80, 81, 82, 83, 83, 84, 84, 85, 86, 87, 88, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 115, 116, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 125, 126, 127, 128, 128, 129, 129, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 142, 143, 144, 145, 146, 147, 148, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 187, 188, 189, 190, 191, 192, 192, 193, 193, 194, 195,

(array([1]),)

After running the language model on the stimuli, reading and saving the hidden states as embeddings:

In [27]:
for layer_id in np.arange(2):
    embeddings = {1:{},2:{},3:{},4:{}}
    for block_id in np.arange(1,5):
        
        print("phrases: ",len(translated_block_texts_steps[block_id]),translated_block_texts_steps[block_id][-1])
        print("scans: ",len(block_scans[block_id]), block_scans[block_id][-1].step)
        count = 0
        for scan in block_scans[block_id]:            
            step = scan.step
            real_step = step
            while real_step not in translated_block_texts_steps[block_id] and (real_step - step) < 3:
                real_step += 1
                print(real_step)
                
            if real_step in translated_block_texts_steps[block_id]:
                count += 1              
                embeddings[block_id][step] = []
                i = 0
                while i < 4 and (real_step+i) in translated_block_texts_steps[block_id]:
                    translated_steps = np.where(np.asarray(translated_block_texts_steps[block_id]) == real_step+i)[0]
                    print("translated steps %s, step %d, word %s" %(translated_steps,step,scan.all_words))
                    one_step_embeddings = []
                    for translated_step in translated_steps:
                        lstm_emb= np.load("../../lm_1b/new_output/subject_"+subject_id+"_block_"+str(block_id)+"_lstm_"+str(layer_id)+"_embeddings/lstm_emb_step_"+str(translated_step+1)+".npy")
                        print(lstm_emb)
                        one_step_embeddings.append(lstm_emb[0])
                    
                    embeddings[block_id][step].append(np.mean(one_step_embeddings,axis=0))
                    print(np.asarray(embeddings[block_id][step]).shape)
                    i += 1
                
        print(count)
        print(len(embeddings[block_id]))
        #print(embeddings[block_id].keys())
    np.save("../embeddings/subject_"+subject_id+"_lstm_"+str(layer_id)+"_emb_objects",embeddings)
    print(len(embeddings[1][0]))

phrases:  1519 1302
scans:  326 1300
translated steps [0], step 0, word ['Harry', 'had', 'never', 'believed']
[[-0.0872765   0.04032643 -0.45139223 ... -0.10783331  0.02813069
  -0.12154386]]
(1, 1024)
translated steps [1], step 0, word ['Harry', 'had', 'never', 'believed']
[[-0.03125925 -0.2477504  -0.03791803 ... -0.40889603  0.07107337
  -0.13472961]]
(2, 1024)
translated steps [2], step 0, word ['Harry', 'had', 'never', 'believed']
[[-0.36215922 -0.9615818  -0.21803743 ...  0.10343758  0.4459945
   0.25937   ]]
(3, 1024)
translated steps [3], step 0, word ['Harry', 'had', 'never', 'believed']
[[-0.8043436  -0.600949    0.3271572  ... -1.068557    0.8109199
  -0.03367356]]
(4, 1024)
translated steps [4], step 4, word ['he', 'would', 'meet', 'a']
[[ 0.4679604  -0.6901602  -0.00552494 ... -0.6177721   0.45266
  -0.21184509]]
(1, 1024)
translated steps [5], step 4, word ['he', 'would', 'meet', 'a']
[[-0.274165   -0.73976    -0.55999136 ... -0.13215657  0.23977607
   0.32054666]]
(2, 10

translated steps [57], step 48, word ['spotted', 'a', 'notice', 'pinned']
[[ 0.6721696  -0.39191964  0.15610366 ... -0.02509441 -0.18976359
   0.15590006]]
(2, 1024)
translated steps [58], step 48, word ['spotted', 'a', 'notice', 'pinned']
[[ 0.49698994  0.5815984  -0.14454858 ... -0.18770427  0.1360281
   0.24327514]]
(3, 1024)
translated steps [59], step 48, word ['spotted', 'a', 'notice', 'pinned']
[[-0.61105853  0.02959094 -0.98691994 ... -0.55249226  0.53511184
  -0.5964625 ]]
(4, 1024)
translated steps [60], step 52, word ['up', 'in', 'the', 'Gryffindor']
[[-0.04075192  0.33440706 -0.14474346 ...  0.09686066 -0.26849535
   0.10120533]]
(1, 1024)
translated steps [61], step 52, word ['up', 'in', 'the', 'Gryffindor']
[[-0.11417457 -0.17248519  0.13726646 ...  0.31120923  0.02301223
  -0.01265299]]
(2, 1024)
translated steps [62], step 52, word ['up', 'in', 'the', 'Gryffindor']
[[ 0.03555911 -0.08425073 -0.15848003 ...  0.00147223  0.23519
   0.06147645]]
(3, 1024)
translated steps 

[[-0.9467751  -0.5083454   0.2535935  ...  0.44005847 -0.59504765
  -0.4290256 ]]
(4, 1024)
translated steps [124], step 108, word ['to', 'learning', 'to', 'fly']
[[-0.16534068 -0.30438778 -0.14773893 ...  0.3821516   0.30692762
  -0.3331754 ]]
(1, 1024)
translated steps [125], step 108, word ['to', 'learning', 'to', 'fly']
[[ 0.14495604 -0.7080718   0.11538798 ... -0.1419447  -0.71135366
   0.67247486]]
(2, 1024)
translated steps [126], step 108, word ['to', 'learning', 'to', 'fly']
[[ 0.9544243  -1.0604056   0.02884674 ... -0.32038265  0.12408245
   0.42519146]]
(3, 1024)
translated steps [127], step 108, word ['to', 'learning', 'to', 'fly']
[[ 0.59173864  0.3690745   1.5099106  ...  0.25765917 -0.3666209
  -0.39049837]]
(4, 1024)
translated steps [128], step 112, word ['more', 'than', 'anything', 'else.']
[[-0.05689004 -0.4330846   0.13381584 ...  0.3200765  -0.01180112
  -0.02250686]]
(1, 1024)
translated steps [129], step 112, word ['more', 'than', 'anything', 'else.']
[[-0.153303

[[ 0.25046057 -0.00950907  0.12766984 ...  0.7131423   0.20379782
  -0.13908078]]
(4, 1024)
translated steps [238], step 208, word ['the', 'countryside', 'on', 'his']
[[ 0.09058475 -0.48425984 -0.2157762  ...  0.4481666   0.34236082
   0.27326214]]
(1, 1024)
translated steps [239], step 208, word ['the', 'countryside', 'on', 'his']
[[-0.9665203   0.8054843  -0.16715986 ...  0.12495594 -0.14918016
   0.36348233]]
(2, 1024)
translated steps [240], step 208, word ['the', 'countryside', 'on', 'his']
[[-0.10636583 -0.27109954 -0.07058778 ...  0.17285992  0.00063538
  -0.29239595]]
(3, 1024)
translated steps [241], step 208, word ['the', 'countryside', 'on', 'his']
[[ 0.27588147 -0.24381045 -0.00248277 ... -0.2831187   0.28302613
   0.080309  ]]
(4, 1024)
translated steps [242 243], step 212, word ['broomstick.', 'Even', 'Ron', 'would']
[[-0.18326658  0.4557421   0.24368502 ...  0.3165254   0.3212681
   0.00805977]]
[[ 0.9179739  -0.03738378  1.1188964  ...  0.7068365   0.1292296
   0.933482

[[-0.6978098  -0.05156871  0.22105974 ... -0.23959602  0.17413826
   0.26104528]]
(2, 1024)
translated steps [302], step 264, word ['a', 'game', 'with', 'only']
[[ 0.25829476  0.3188326   0.03917577 ... -0.20601954  0.7206993
   0.00192336]]
(3, 1024)
translated steps [303], step 264, word ['a', 'game', 'with', 'only']
[[ 0.60936344 -0.63270444  0.323876   ... -0.49292994  0.0531789
   0.250764  ]]
(4, 1024)
translated steps [304], step 268, word ['one', 'ball', 'where', 'no']
[[ 0.6531505  -0.4628579  -0.21341044 ...  0.0920537   0.23662484
  -0.26703244]]
(1, 1024)
translated steps [305], step 268, word ['one', 'ball', 'where', 'no']
[[-0.35353813  0.1431164   0.8549155  ... -0.4080735   0.67440164
   0.13217965]]
(2, 1024)
translated steps [306], step 268, word ['one', 'ball', 'where', 'no']
[[ 0.40446067 -0.19218642  0.9690692  ... -0.47784066  0.23686147
   0.17130283]]
(3, 1024)
translated steps [307], step 268, word ['one', 'ball', 'where', 'no']
[[ 0.08977976 -0.53471637  0.069

...

   0.10830657]]
(3, 1024)
translated steps [1351], step 4829, word ['the', 'corridor.', 'Filch', 'must']
[[-0.0300394  -0.31500807 -0.15732692 ... -0.17481671 -0.6374938
  -0.45979193]]
(4, 1024)
translated steps [1352], step 4833, word ['have', 'hurried', 'off', 'to']
[[-0.03014204 -0.35658556  0.07687773 ... -0.14989114 -0.17560834
  -0.11542773]]
(1, 1024)
translated steps [1353], step 4833, word ['have', 'hurried', 'off', 'to']
[[ 0.11107874 -0.05683271 -0.06234964 ...  0.00286774 -0.03598696
  -0.32747096]]
(2, 1024)
translated steps [1354], step 4833, word ['have', 'hurried', 'off', 'to']
[[ 0.4931564   0.17653057 -0.9059225  ...  0.11297314  0.60138404
  -0.81025326]]
(3, 1024)
translated steps [1355], step 4833, word ['have', 'hurried', 'off', 'to']
[[ 0.58832645  0.55184084 -0.20737222 ... -0.50986445  0.29405442
  -0.84237766]]
(4, 1024)
translated steps [1356], step 4837, word ['look', 'for', 'them', 'somewhere']
[[ 0.6772089  -0.03997524 -0.33945233 ... -0.3873056  -0.39728

[[ 1.0453763  -0.6282016   0.66460377 ...  1.0625482  -0.5272524
   0.41296014]]
(3, 1024)
translated steps [1426], step 4897, word ['looking', 'at', 'their', 'bathrobes']
[[ 0.44619423  0.26057333 -0.22977357 ...  1.0414084  -0.27210075
  -0.37634632]]
(4, 1024)
translated steps [1427], step 4901, word ['hanging', 'off', 'their', 'shoulders']
[[-0.197312    0.6442141  -0.09384928 ...  0.6654151  -0.37235114
  -0.14640772]]
(1, 1024)
translated steps [1428], step 4901, word ['hanging', 'off', 'their', 'shoulders']
[[ 0.8247079   0.06900364  0.7073356  ...  1.1477509  -0.2593845
  -0.666853  ]]
(2, 1024)
translated steps [1429], step 4901, word ['hanging', 'off', 'their', 'shoulders']
[[ 0.47445112 -0.94261134  0.99295145 ...  0.63596237 -0.05030236
  -0.02788519]]
(3, 1024)
translated steps [1430], step 4901, word ['hanging', 'off', 'their', 'shoulders']
[[ 0.15719728  0.2620984  -0.13893226 ...  0.9288141   0.13399723
  -0.454415  ]]
(4, 1024)
translated steps [1431], step 4905, word 

[[-0.14763695 -0.25375655  0.14067802 ...  0.4459178   0.38358825
  -0.48134297]]
(1, 1024)
translated steps [1503], step 4961, word ['think', "they're", 'doing,', 'keeping']
[[-0.13078767 -0.19086224  0.51877373 ...  0.04737906  0.39381644
  -0.11485597]]
(2, 1024)
translated steps [1504 1505], step 4961, word ['think', "they're", 'doing,', 'keeping']
[[ 0.43736604 -0.5590202  -0.05498081 ... -0.08471328  0.06746531
  -0.20957543]]
[[-0.29904482  0.3121993   0.74463177 ... -0.26894554  0.21329965
  -0.09523972]]
(3, 1024)
translated steps [1506], step 4961, word ['think', "they're", 'doing,', 'keeping']
[[ 0.02652352  0.15151286  0.79773283 ... -0.65129757  0.41704595
  -0.31857488]]
(4, 1024)
translated steps [1507], step 4965, word ['a', 'thing', 'like', 'that']
[[ 0.47555453 -0.04754731  0.7013588  ... -0.2035354   0.06824035
  -0.12570325]]
(1, 1024)
translated steps [1508], step 4965, word ['a', 'thing', 'like', 'that']
[[ 0.34150663 -0.07160688  0.5356426  ... -0.57952607  0.081

[[-0.06141563 -0.15440503  0.06963784 ...  0.06133284 -0.24072859
  -0.05736036]]
(2, 1024)
translated steps [1617 1618], step 5049, word ['obviously', 'guarding', 'something."', '+']
[[ 0.37638593 -0.13413203  0.09299478 ...  0.10202233 -0.20539682
   0.02394659]]
[[-0.05004114 -0.04607616 -0.0538953  ... -0.21525526 -0.507263
  -0.2285707 ]]
(3, 1024)
translated steps [1619], step 5049, word ['obviously', 'guarding', 'something."', '+']
[[-0.02614896  0.21138282 -0.54223394 ... -0.29709616 -0.06896374
   0.38886517]]
(4, 1024)
translated steps [1620], step 5053, word ['She', 'stood', 'up,', 'glaring']
[[ 0.10486694 -0.24977559  0.12887743 ...  0.0303594   0.44216654
   0.42820963]]
(1, 1024)
translated steps [1621], step 5053, word ['She', 'stood', 'up,', 'glaring']
[[ 0.30870792 -0.2430895  -0.4168709  ... -0.408452   -0.24877833
  -0.59979135]]
(2, 1024)
translated steps [1622 1623], step 5053, word ['She', 'stood', 'up,', 'glaring']
[[ 0.5646103  -0.24947661 -0.79915434 ... -0.002

[[-0.19873317 -0.26017532  0.43584967 ...  0.07847368  0.16088746
   0.31069848]]
(2, 1024)
translated steps [1735], step 5145, word ['something', 'you', 'wanted', 'to']
[[ 0.54277897 -0.39071512  0.76187706 ...  0.06471598  0.5767339
   0.0810885 ]]
(3, 1024)
translated steps [1736], step 5145, word ['something', 'you', 'wanted', 'to']
[[ 0.3642405  -0.00939201  0.4770856  ... -0.12880506  0.31757498
   0.12237018]]
(4, 1024)
translated steps [1737], step 5149, word ['hide', '--', 'except', 'perhaps']
[[ 0.74330086 -0.60876125 -0.4377377  ...  0.47757486  0.3983637
  -0.1265132 ]]
(1, 1024)
translated steps [1738], step 5149, word ['hide', '--', 'except', 'perhaps']
[[-0.05464683 -0.3189904   1.1977025  ... -0.34878182  0.2534064
  -0.18875197]]
(2, 1024)
translated steps [1739], step 5149, word ['hide', '--', 'except', 'perhaps']
[[ 0.19300635 -0.11305723  0.7017882  ... -0.2499704   0.5169721
  -0.09352225]]
(3, 1024)
translated steps [1740], step 5149, word ['hide', '--', 'except',

In [28]:
vocab_set = list(set(vocab))

In [29]:
len(vocab_set)

1837

In [30]:
#read words:
with open("subject_"+subject_id+"_harry_potter_vocab.txt","r") as fin:
    print("Reading words...")
    words = fin.readlines()
    for i in np.arange(len(words)):
        words[i] = words[i].replace('\n',"")

Reading words...


In [32]:
word_embeddings = np.load("../embeddings/subject_1/embeddings_char_cnn.npy")#("embeddings_char_cnn.npy")
len(word_embeddings)

5176

In [33]:
word_embedding_dic = {}
for i in np.arange(len(words)):
    word_embedding_dic[words[i]] = word_embeddings[i]
    
np.save("../embeddings/subject_"+subject_id+"_char_cnn_embedding_dic",word_embedding_dic)

In [35]:
softmax_embeddings = np.load("../embeddings/subject_1/embeddings_softmax.npy")

In [36]:
softmax_embeddings.shape

(800000, 1024)

In [37]:
all_vocab = []
softmax_vocab = []
with open("../../lm_1b/data/vocab-2016-09-10.txt","r") as vf:
    lines = vf.readlines()
    for line in lines:
        softmax_vocab.append(line.replace('\n',''))

In [38]:
softmax_word_embedding_dic = {}
for i in np.arange(len(words)):
    if words[i] in softmax_vocab:
        softmax_word_embedding_dic[words[i]] = softmax_embeddings[softmax_vocab.index(words[i])]
    elif words[i].lower() in softmax_vocab:
        softmax_word_embedding_dic[words[i]] = softmax_embeddings[softmax_vocab.index(words[i].lower())]
    else:
        softmax_word_embedding_dic[words[i]] = softmax_embeddings[softmax_vocab.index('<UNK>')]
        
np.save("../embeddings/subject_"+subject_id+"_softmax_embedding_dic",word_embedding_dic)

In [39]:
coords[:,0]

array([ 3,  3,  3, ..., 51, 51, 51], dtype=uint8)

In [40]:
brain3d = np.zeros((np.max(coords[:,0])+1,np.max(coords[:,1])+1,np.max(coords[:,2])+1))

In [41]:
block_scans[1][2]

In [42]:
block_id = 1
for k in np.arange(len(block_scans[block_id])):
    brain3d = np.zeros((np.max(coords[:,0])+1,np.max(coords[:,1])+1,np.max(coords[:,2])+1))    
    for i in np.arange(len(coords)):
        brain3d[coords[i][0],coords[i][1],coords[i][2]] = block_scans[block_id][k].activations[0][i]
    block_scans[block_id][k].brain3d = brain3d

In [43]:
x,y,z = block_scans[1][0].brain3d.nonzero()

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, -z, zdir='z', c= 'red')
plt.savefig("demo.png")


In [44]:
np.save(processed_data_dir+"/subject_"+subject_id+"_extended_scan_object",block_scans)

In [45]:
loaded_extended_scans = np.load(processed_data_dir+"/subject_"+subject_id+"_extended_scan_object.npy")

print(loaded_extended_scans.item().keys())
block_id = 1
print(len(loaded_extended_scans.item()[block_id]))
i = 0
print(loaded_extended_scans.item()[block_id][i].all_words)

dict_keys([1, 2, 3, 4])
326
['Harry', 'had', 'never', 'believed']


In [46]:
scan_objects = np.load(processed_data_dir+"/subject_"+subject_id+"_scan_objects.npy")
glove_embeddings = np.load("../embeddings/glove_word_embedding_dic.npy")
glove_embeddings.item()[""] = np.zeros_like(glove_embeddings.item()[list(glove_embeddings.item().keys())[0]])
glove_embeddings.item()["+"] = np.zeros_like(glove_embeddings.item()[list(glove_embeddings.item().keys())[0]])

In [48]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

block_ids = [1,2,3,4]

filtered_glove_embedding_step_dic = {1:{},2:{},3:{},4:{}}
for block_id in block_ids:
    for scan_obj in scan_objects.item().get(block_id):
        words = []
        current_glove_words = []
        current_glove_embeddings = []
        for wordgroup in scan_obj.current_translated_words:
            words.extend(wordgroup)
                    
        if set(words).issubset(glove_embeddings.item().keys()):
            current_glove_words =  words
        else:
            for w in words:
                if w in glove_embeddings.item().keys():
                    current_glove_words.append(w)
                elif w.lower() in glove_embeddings.item().keys():
                    current_glove_words.append(w.lower())
                else:
                    tokenized_w = nltk.word_tokenize(w.lower())
                    if set(tokenized_w).issubset(glove_embeddings.item().keys()):
                        current_glove_words.extend(tokenized_w)
                    else:
                        for t_w in tokenized_w:
                            if t_w in glove_embeddings.item().keys():
                                current_glove_words.append(t_w)
                            else:
                                l_t_w = lemmatizer.lemmatize(t_w)
                                if l_t_w in glove_embeddings.item().keys():
                                    current_glove_words.append(t_w)
                                else:
                                    s_l_t_w = ps.stem(l_t_w)
                                    if s_l_t_w in glove_embeddings.item().keys():
                                        current_glove_words.append(s_l_t_w)
                                    else:
                                        print(w)
        for w in current_glove_words:
            current_glove_embeddings.append(glove_embeddings.item()[w])
            
        filtered_glove_embedding_step_dic[block_id][scan_obj.step] = (current_glove_words,current_glove_embeddings)
            

gloatingly
Remembrall
Remembrall
Remembrall
Remembrall
Remembrall
Remembrall
Remembrall
Pomfrey
Ickle
Firsties
caughty
Alohomora
haaa
Haaaaaa


In [49]:
np.save("filtered_glove_embedding_step_dic",filtered_glove_embedding_step_dic)